In [10]:
# Import Python package
import binhosupernova
from binhosupernova.supernova import Supernova
from binhosupernova.commands.i3c.definitions import *

## Getting started

#### 1. List all the Supernova devices connected to the PC host

The ``binhosupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [11]:
binhosupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#6&13ea2591&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': 'EC68DCA0BE439450A60D391C6FDC1D3D',
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'hardware_version': 'C',
  'firmware_version': '4.1.2'}]

#### 2. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [12]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 3. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``binhosupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [13]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [14]:
# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:

    if supernova_message != None:

        # Print a header
        print(">> New message from SUPERNOVA:")
        print(supernova_message)

    if system_message != None:

        # Print a header
        print(">> New message from the SYSTEM:")
        print(system_message)

In [15]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [16]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 6. Test communication with a basic command such as ``GET DEVICE INFO``

In [17]:
supernova.getDeviceInfo(getId())

{'module': 1, 'opcode': 0, 'message': 'GET DEVICE INFO request success'}

>> New message from SUPERNOVA:
{'id': 1, 'command': 'SYS GET DEVICE INFO', 'result': 'SUCCESS', 'manufacturer': 'Binho LLC', 'product_name': 'Supernova', 'serial_number': 'EC68DCA0BE439450A60D391C6FDC1D3D', 'hardware_version': 'C', 'firmware_version': '4.1.2-a9d09499', 'capabilities': {'supported_groups': ['I3C', 'I2C', 'SPI', 'UART', 'GPIO']}}


## I3C Protocol API

The I3C Protocol API methods are described below.

### 1. Configure the Supernova as an I3C Controller

``Supernova.i3cControllerInit(id, pushPullRate, i3cOpenDrainRate, i2cOpenDrainRate)``

This method initializes the Supernova as an I3C controller.

  - `id: int`: A 2-byte integer that represents the transfer ID.
  - `pushPullRate: I3cPushPullTransferRate`: Push-Pull frequency.
  - `i3cOpenDrainRate: I3cOpenDrainTransferRate`: I3C frequency in Open Drain mode.
  - `i2cOpenDrainRate: I2cTransferRate`: I2C frequency.

In [18]:
supernova.i3cControllerInit(getId(), I3cPushPullTransferRate.PUSH_PULL_3_125_MHZ_31_25_DC, I3cOpenDrainTransferRate.OPEN_DRAIN_1_MHZ, I2cTransferRate._400KHz)

{'module': 1, 'opcode': 0, 'message': 'I3C CONTROLLER INIT request success'}

>> New message from SUPERNOVA:
{'id': 2, 'command': 'I3C CONTROLLER INIT', 'result': 'SUCCESS'}


### 2. Set I3C voltage

``Supernova.setI3cVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the I3C bus in mV, ranging from 800 mV up to 3300 mV. 

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [800, 3300] mV and allows the value 0 mV to power off the output voltage.

There are 2 I3C ports:
- I3C LV: from 800 mV to 1199 mV
- I3C HV: from 1200 mV to 3300 mV

In [19]:
supernova.setI3cVoltage(getId(), 3300)

{'module': 1, 'opcode': 0, 'message': 'SET I3C VOLTAGE request success'}

>> New message from SUPERNOVA:
{'id': 3, 'command': 'SYS SET I3C VOLTAGE', 'result': 'SUCCESS'}


### 3. I3C bus initialization

There are two options:
1. Initialize the bus with an **empty target devices table** and scan the I3C bus.
2. Add target to the target device table before initializing the bus.

In this case, the I3C Controller performs an RSTDAA and ENTDAA CCC. If one or more I3C Targets acknowledge the Broadcast Address 0x7E, then the Supernova 
will assign the lower available and valid dynamic address.

Previously, retrieve the target devices table with the method: `Supernova.i3cControllerGetTargetDevicesTable(getId())`

In [20]:
# Get the Target Devices Table from the Supernova. It must be empty before the bus initialization.
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 4, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 0, 'table': []}


`Supernova.i3cControllerInitBus(id, table)`

- `id: int`: A 2-byte integer that represents the transfer ID.
- `table: dict`: A Python dictionary containing information about the target devices table. Default value: `None`

In [21]:
# Initialize and scan the I3C bus.
request_result = supernova.i3cControllerInitBus(getId())

>> New message from SUPERNOVA:
{'id': 5, 'command': 'I3C CONTROLLER INIT BUS', 'result': 'SUCCESS', 'invalid_addresses': []}


In [22]:
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 6, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 1, 'table': [{'static_address': 0, 'dynamic_address': 8, 'pid': [4, 106, 0, 0, 0, 0], 'bcr': 39, 'dcr': 160, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}]}


## I3C HDR-DDR WRITE

`Supernova.i3cControllerHdrDdrWrite(self, id: c_uint16, targetAddress: c_uint8, command: c_uint8, data: list)`

- `id: int`: A 2-byte unsigned integer representing the transfer ID. The range allowed is [1, 65535].
- `targetAddress`: The dynamic address of the target device for writing data.
- `command`: the HDR-DDR write command that is sent as part of the HDR-DDR Command Word. It must be in the range [0x00, 0x7F].
- `data`: A list containing the data to be written to the device. The length of this list must be a multiple of 2.

1. Validate the value passed as the command.

In [23]:
target_address = 0x08
command = 0x85                      # Write command: 8'h00 to 8'h7F
data = [i for i in range(0, 20)]

supernova.i3cControllerHdrDdrWrite(getId(), target_address, command, data)

{'module': 1,
 'opcode': 1,
 'message': 'ARGUMENT ERROR: the HDR-DDR command in write transfers must be in the range [0x00, 0x7F]'}

2. Validate the length of the data.

In [24]:
target_address = 0x08
command = 0x05                      # Write command: 8'h00 to 8'h7F.
data = [i for i in range(0, 7)]     # Length must be multiple of 2.

supernova.i3cControllerHdrDdrWrite(getId(), target_address, command, data)

{'module': 1,
 'opcode': 1,
 'message': 'ARGUMENT ERROR: data array length must be even in the range [2, 1024]'}

3. All the parameters are correctly validated.

In [25]:
target_address = 0x08
command = 0x04                       # Write command: 8'h00 to 8'h7F.
data = [i for i in range(0, 18)]     # Length must be multiple of 2.

supernova.i3cControllerHdrDdrWrite(getId(), target_address, command, data)

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER PRIVATE WRITE request success'}

>> New message from SUPERNOVA:
{'id': 9, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 19}


In [26]:
target_address = 0x08
command = 0x04                      # Write command: 8'h00 to 8'h7F.
data = [i for i in range(0, 60)]     # Length must be multiple of 2.

supernova.i3cControllerHdrDdrWrite(getId(), target_address, command, data)

>> New message from SUPERNOVA:
{'id': 10, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 61}


{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER PRIVATE WRITE request success'}

## I3C HDR-DDR READ

`Supernova.i3cControllerHdrDdrRead(self, id: c_uint16, targetAddress: c_uint8, command: c_uint8, length: c_uint16)`

- `id: int`: A 2-byte unsigned integer representing the transfer ID. The range allowed is [1, 65535].
- `targetAddress`: The dynamic address of the target device.
- `command`: the HDR-DDR read command that is sent as part of the HDR-DDR Command Word. It must be in the range [0x80, 0xFF].
- `length`: The length of the data to be read. The length must be multiple of 2.

1. Validate the value passed as the command.

In [27]:
target_address = 0x08
command = 0x03                      # Read command: 8'h80 to 8'hFF
length = 18

supernova.i3cControllerHdrDdrRead(getId(), target_address, command, length)

{'module': 1,
 'opcode': 1,
 'message': 'ARGUMENT ERROR: the HDR-DDR command in read transfers must be in the range [0x80, 0xFF]'}

2. Validate the length of the data.

In [28]:
target_address = 0x08
command = 0x80                      # Read command: 8'h80 to 8'hFF
length = 17

supernova.i3cControllerHdrDdrRead(getId(), target_address, command, length)

{'module': 1,
 'opcode': 1,
 'message': 'ARGUMENT ERROR: data length must be even and in the range [2, 1024]'}

3. All the parameters are set correctly.

In the cell below, an I3C SDR Private Write Transfer is run to reset the internal register address of the ICM426xx target to 0x00 and read back the data written using the method `i3cControllerHdrDddWrite` described above.

In [29]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
REG_ADDR        = [0x00]
DATA            = []        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
supernova.i3cControllerWrite(getId(),TARGET_ADDR,MODE,REG_ADDR,DATA,startWith7E=True)

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER PRIVATE WRITE request success'}

>> New message from SUPERNOVA:
{'id': 13, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 0}


In [30]:
target_address = 0x08
command = 0x80                      # Read command: 8'h80 to 8'hFF
length = 16

supernova.i3cControllerHdrDdrRead(getId(), target_address, command, length)

>> New message from SUPERNOVA:
{'id': 14, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 16, 'payload': [1, 0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]}


{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER PRIVATE READ request success'}

Set the MRL to 20 bytes.

In [31]:
supernova.i3cDirectSETMRL(getId(), 0x08, 20)

>> New message from SUPERNOVA:

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}


{'id': 15, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETMRL', 'payload_length': 2}


Try to read 30 bytes from the target, but only 20 are returned by the target because of the SETMRL set above.

In [32]:
target_address = 0x08
command = 0x80                      # Read command: 8'h80 to 8'hFF
length = 30

supernova.i3cControllerHdrDdrRead(getId(), target_address, command, length)

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER PRIVATE READ request success'}

>> New message from SUPERNOVA:
{'id': 16, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 20, 'payload': [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 0, 0, 128, 0, 128, 0, 128, 0, 128]}


In [33]:
supernova.i3cControllerTriggerHdrExitPattern(getId())

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER TRIGGER PATTERN request success'}

>> New message from SUPERNOVA:
{'id': 17, 'command': 'I3C CONTROLLER TRIGGER PATTERN', 'result': 'SUCCESS', 'pattern': 'I3C_HDR_EXIT_PATTERN'}


## I3C HDR-DDR CCCs

As of now, the Supernova SDK does not include explicit methods for issuing Common Command Codes (CCCs) in HDR-DDR mode. Since CCCs in HDR-DDR mode are less common than in SDR mode, a generic method is provided to issue all CCCs in HDR-DDR mode. Examples of using this generic method are shown below.

#### HDR-DDR Broadcast DISEC (0x01)

In [34]:
supernova.i3cControllerCccTransfer(id=getId(),
                                   cmdType=I3cCccType.CCC_WITH_DEFINING_BYTE,
                                   direction=TransferDirection.WRITE,
                                   targetAddress=0x7E,
                                   mode=TransferMode.I3C_HDR_DDR,
                                   defByte=0X00,                    # This value must be 0x00 if the CCC does not have a defining byte.
                                   ccc=CCC.B_DISEC,
                                   length=1,
                                   data= [DISEC.DISINT.value | DISEC.DISHJ.value])

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}

>> New message from SUPERNOVA:
{'id': 18, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_DISEC', 'payload_length': 3}


#### HDR-DDR Broadcast ENEC (0x01)

In [35]:
supernova.i3cControllerCccTransfer(id=getId(),
                                   cmdType=I3cCccType.CCC_WITH_DEFINING_BYTE,
                                   direction=TransferDirection.WRITE,
                                   targetAddress=0x7E,
                                   mode=TransferMode.I3C_HDR_DDR,
                                   defByte=0X00,                    # This value must be 0x00 if the CCC does not have a defining byte.
                                   ccc=CCC.B_ENEC,
                                   length=1,
                                   data= [ENEC.ENINT.value])

>> New message from SUPERNOVA:

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}


{'id': 19, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_ENEC', 'payload_length': 3}


#### HDR-DDR Direct DISEC (0x01)

In [36]:
supernova.i3cControllerCccTransfer(id=getId(),
                                   cmdType=I3cCccType.CCC_WITH_DEFINING_BYTE,
                                   direction=TransferDirection.WRITE,
                                   targetAddress=0x08,
                                   mode=TransferMode.I3C_HDR_DDR,
                                   defByte=0X00,                    # This value must be 0x00 if the CCC does not have a defining byte.
                                   ccc=CCC.D_DISEC,
                                   length=1,
                                   data= [DISEC.DISINT.value | DISEC.DISHJ.value])

>> New message from SUPERNOVA:

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}


{'id': 20, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_DISEC', 'payload_length': 1}


#### HDR-DDR Direct ENEC (0x01)

In [37]:
supernova.i3cControllerCccTransfer(id=getId(),
                                   cmdType=I3cCccType.CCC_WITH_DEFINING_BYTE,
                                   direction=TransferDirection.WRITE,
                                   targetAddress=0x08,
                                   mode=TransferMode.I3C_HDR_DDR,
                                   defByte=0X00,                    # This value must be 0x00 if the CCC does not have a defining byte.
                                   ccc=CCC.D_ENEC,
                                   length=1,
                                   data= [ENEC.ENINT.value])

>> New message from SUPERNOVA:

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}


{'id': 21, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_ENEC', 'payload_length': 1}


#### HDR-DDR - Broadcast SETMRL (0x0A)

In [38]:
supernova.i3cControllerCccTransfer(id=getId(),
                                   cmdType=I3cCccType.CCC_WITHOUT_DEFINING_BYTE,
                                   direction=TransferDirection.WRITE,
                                   targetAddress=0x7E,
                                   mode=TransferMode.I3C_HDR_DDR,
                                   defByte=0X00,                    # This value must be 0x00 if the CCC does not have a defining byte.
                                   ccc=CCC.B_SETMRL,
                                   length=3,
                                   data=[0x01, 0x02, 0x03])

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}

>> New message from SUPERNOVA:
{'id': 22, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_SETMRL', 'payload_length': 5}


#### HDR-DDR Broadcast RSTACT (0x2A)

In [39]:
supernova.i3cControllerCccTransfer(id=getId(),
                                   cmdType=I3cCccType.CCC_WITH_DEFINING_BYTE,
                                   direction=TransferDirection.WRITE,
                                   targetAddress=0x7E,
                                   mode=TransferMode.I3C_HDR_DDR,
                                   defByte=I3cTargetResetDefByte.RESET_I3C_PERIPHERAL.value,                    # This value must be 0x00 if the CCC does not have a defining byte.
                                   ccc=CCC.B_RSTACT,
                                   length=0,
                                   data=[])

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}

>> New message from SUPERNOVA:
{'id': 23, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_RSTACT', 'payload_length': 2}


#### HDR-DDR Direct SETMRL (0x8A)

In [40]:
supernova.i3cControllerCccTransfer(id=getId(),
                                   cmdType=I3cCccType.CCC_WITHOUT_DEFINING_BYTE,
                                   direction=TransferDirection.WRITE,
                                   targetAddress=0x08,
                                   mode=TransferMode.I3C_HDR_DDR,
                                   defByte=0x00,                    # This value must be 0x00 if the CCC does not have a defining byte.
                                   ccc=CCC.D_SETMRL,
                                   length=2,
                                   data=[0xAB,0xCD])

>> New message from SUPERNOVA:

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}


{'id': 24, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETMRL', 'payload_length': 2}


#### HDR-DDR Direct GETMRL (0x8C)

In [41]:
supernova.i3cControllerCccTransfer(id=getId(),
                                   cmdType=I3cCccType.CCC_WITHOUT_DEFINING_BYTE,
                                   direction=TransferDirection.READ,
                                   targetAddress=0x08,
                                   mode=TransferMode.I3C_HDR_DDR,
                                   defByte=0x00,                    # This value must be 0x00 if the CCC does not have a defining byte.
                                   ccc=CCC.D_GETMRL,
                                   length=2,
                                   data=[])

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}

>> New message from SUPERNOVA:
{'id': 25, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMRL', 'payload_length': 2, 'payload': [0, 16]}


In [42]:
supernova.i3cGETMRL(getId(), 0x08)

>> New message from SUPERNOVA:
{'id': 26, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMRL', 'payload_length': 2, 'payload': [0, 20]}


{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}

#### HDR-DDR Broadcast SETMRL (0x8A)

In [43]:
supernova.i3cControllerCccTransfer(id=getId(),
                                   cmdType=I3cCccType.CCC_WITHOUT_DEFINING_BYTE,
                                   direction=TransferDirection.WRITE,
                                   targetAddress=0x7E,
                                   mode=TransferMode.I3C_HDR_DDR,
                                   defByte=0x00,                    # This value must be 0x00 if the CCC does not have a defining byte.
                                   ccc=CCC.B_SETMRL,
                                   length=2,
                                   data=[0xAB,0xCD])

{'module': 1, 'opcode': 0, 'message': 'I3C SEND CCC request success'}

>> New message from SUPERNOVA:
{'id': 27, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_SETMRL', 'payload_length': 4}


### Reset I3C BUS

Let's reset the bus to initialize the bus again, this time sending a non-empty target devices table to the Supernova.

`Supernova.i3cControllerResetBus(id)`

This method requests the Supernova to: 1. issue an RSTDAA CCC, 2. free the already assigned dynamic addresses and 3. clean up the target devices table. 

- `id: int`: A 2-byte integer that represents the transfer ID.

In [44]:
supernova.i3cControllerResetBus(getId())

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER RESET BUS request success'}

>> New message from SUPERNOVA:
{'id': 28, 'command': 'I3C CONTROLLER RESET BUS', 'result': 'SUCCESS'}


In [45]:
# The target devices table must be empty again after resetting the bus.
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 29, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 0, 'table': []}


### Close communication

Use the ``Supernova.close()`` method to end the communication with the Supernova device and release the used memory in the background like threads and so on.

In [46]:
# Close the communication with the Supernova device.
supernova.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}